In [4]:
import pandas as pd
from tqdm import tqdm
import os
import yaml

import sys
sys.path.append('../mlrose')
from mlrose_hiive.generators.four_peaks_generator import FourPeaksGenerator
from mlrose_hiive import RHCRunner

In [5]:
ALGORITHM = 'rhc'
MUTATION_RATE_LIST = [0.25, 0.5, 0.75, 0.9]

T_PCT = [0.25]
PROBLEM_SIZE_LIST = [5, 10, 20, 50, 100, 200, 500, 1000]
ITERATIONS_LIST = [1000]
MAX_ATTEMPTS_LIST = [1000]
NUM_RUNS = [3]
POPULATION_SIZES_LIST = [10, 50, 100, 200, 500, 1000]

In [6]:
from mlrose_hiive.fitness.four_peaks import FourPeaks
print(500*0.498)
problem = FourPeaks(t_pct=0.1)
problem.evaluate([1]*13 + [0]*37)

249.0


87

In [53]:
max_attempts = 10000
iterations = 10000
restarts = 5
problem_size = 20
t_pct = 0.2

In [54]:
def get_opt_fitness(problem_size):
    R = (1- t_pct)*problem_size
    if R == int(R):
        R = int(R) - 1
    else:
        R = int(R)

    return problem_size + R
OPT_FITNESS = get_opt_fitness(problem_size)
print(OPT_FITNESS)

35


In [55]:
# SEED = 6
import time
SEED = int(time.time())
problem = FourPeaksGenerator().generate(seed=SEED, size=problem_size, t_pct=t_pct)
runner = RHCRunner(problem=problem,
                    experiment_name='dontcare',
                    output_directory='.',
                    seed=SEED,
                    iteration_list=[iterations],
                    max_attempts=max_attempts,
                    restart_list=[restarts],
                    generate_curves=True)
x, df_run_curves = runner.run()

In [56]:
df_run_curves['Fitness'].max()

20.0

In [57]:
df_run_curves.tail(10)

,Iteration,Time,Fitness,FEvals,Restarts,max_iters,current_restart
59991,9991,16.167588,20.0,60034.0,5,10000,5
59992,9992,16.170960,20.0,60035.0,5,10000,5
59993,9993,16.174444,20.0,60036.0,5,10000,5
59994,9994,16.177655,20.0,60037.0,5,10000,5
59995,9995,16.180840,20.0,60038.0,5,10000,5
59996,9996,16.183393,20.0,60039.0,5,10000,5
59997,9997,16.185879,20.0,60040.0,5,10000,5
59998,9998,16.188675,20.0,60041.0,5,10000,5
59999,9999,16.191248,20.0,60042.0,5,10000,5
60000,10000,16.194808,20.0,60043.0,5,10000,5


In [ ]:
df_path = f'metrics/{ALGORITHM}.csv'
if not os.path.exists(df_path):
    df = pd.DataFrame(columns=[
        'problem_size', 'mutation_rate', 'population_size', 'iterations', 'max_attempts', 
        'run_number', 'best_fitness', 'time', 'fevals'])
    df.to_csv(df_path, index=False)
df = pd.read_csv(df_path)

In [ ]:
all_df = pd.DataFrame()
group_i = 0
run_i = 0
for problem_size in PROBLEM_SIZE_LIST:
    print(f'Problem Size: {problem_size}')
    for iterations in ITERATIONS_LIST:
        print(f'Iterations: {iterations}')
        for max_attempts in MAX_ATTEMPTS_LIST:
            print(f'Max Attempts: {max_attempts}')
            for population_size in POPULATION_SIZES_LIST:
                print(f'Population Size: {population_size}')
                for mutation_rate in MUTATION_RATE_LIST:
                    print(f"Mutation Rate: {mutation_rate}")
                    for i in tqdm(range(NUM_RUNS)):
                        problem = FourPeaksGenerator().generate(seed=SEED+i, size=problem_size, t_pct=T_PCT)
                        sa = GARunner(
                            problem=problem,
                            experiment_name='dontcare',
                            output_directory='./',
                            seed=SEED+i,
                            max_attempts=max_attempts,
                            iteration_list=[iterations],
                            population_sizes=[population_size],
                            mutation_rates=[mutation_rate]
                        )
                        _, df_run_curves = sa.run()
                        df_run_curves['group_number'] = group_i
                        df_run_curves['run_number'] = run_i
                        df_run_curves['problem_size'] = problem_size
                        df_run_curves['max_iterations'] = iterations
                        df_run_curves['max_attempts'] = max_attempts
                        df_run_curves['population_size'] = population_size
                        df_run_curves['mutation_rate'] = mutation_rate

                        print(f"Max Fitness: {df_run_curves['Fitness'].max()}")
                        print(f"Max Iteration: {df_run_curves['Iteration'].max()}")

                        all_df = pd.concat([all_df, df_run_curves], axis=0)
                        run_i += 1
                    group_i += 1
all_df.reset_index(inplace=True, drop=True)

In [ ]:
print(f"Max: {all_df['Fitness'].max()}")
print(f"Min: {all_df['Fitness'].min()}")

In [ ]:
all_df.to_csv(os.path.join(output_dir, 'learning_curve.csv'), index=False)